In [16]:
import pandas as pd
from sqlalchemy import create_engine
#from config import db_pwd
import os
import sys
os.getcwd()
sys.path.append("C:/SQL_PW/")
import config

### Store CSV into DataFrame

In [17]:
csv_file = "Resources/snap_national_data.csv"
snap_df = pd.read_csv(csv_file)
snap_df.head()

,﻿FIPS,FIPSTXT,COFIPS,CODEF,CONAME,STATE_CD,STFIPS,URBCODE,METRO,POP00,...,PRGBEN02,PRGBEN03,PRGBEN04,PRGBEN05,PRGBEN06,PRGBEN07,PRGBEN08,PRGBEN09,PRGBEN10,PRGBEN11
0,31025.0,31025.0,25.0,CEN_BEA,"Cass County, NE",NE,31.0,2.0,1.0,24334.0,...,813.0,911.0,1107.0,1173.0,1059.0,1083.9,1257.1,1955.2,2520.0,-9991.0
1,31069.0,31069.0,69.0,CEN_BEA,"Garden County, NE",NE,31.0,9.0,0.0,2292.0,...,72.0,83.0,86.0,103.0,116.0,119.5,142.1,246.6,272.9,-9991.0
2,31073.0,31073.0,73.0,CEN_BEA,"Gosper County, NE",NE,31.0,9.0,0.0,2143.0,...,35.8,28.8,36.4,52.3,78.0,63.9,93.2,142.1,138.6,-9991.0
3,31113.0,31113.0,113.0,CEN_BEA,"Logan County, NE",NE,31.0,9.0,0.0,774.0,...,8.0,28.0,9.6,39.2,54.0,19.9,57.1,89.8,99.2,-9991.0
4,31137.0,31137.0,137.0,CEN_BEA,"Phelps County, NE",NE,31.0,7.0,0.0,9747.0,...,320.0,412.0,548.0,557.0,548.0,557.7,658.3,826.8,1021.4,-9991.0


### Clean DataFrame

In [18]:
new_snap_df = snap_df[['CODEF', 'STATE_CD', 'POP10', 'NUMPOV10', 'PRGNUM10', 'PRGBEN10']].copy()
new_snap_df.head()

,CODEF,STATE_CD,POP10,NUMPOV10,PRGNUM10,PRGBEN10
0,CEN_BEA,NE,25.291,1.787,1.727,2520.0
1,CEN_BEA,NE,2.065,321.000,187.000,272.9
2,CEN_BEA,NE,2.047,191.000,95.000,138.6
3,CEN_BEA,NE,770.000,96.000,68.000,99.2
4,CEN_BEA,NE,9.199,894.000,700.000,1021.4


### Connect to local database

In [19]:
rds_connection_string = f"root:{config.password}@127.0.0.1:3306/snap_db"
engine = create_engine(f'mysql://{rds_connection_string}')

engine.table_names()

['snap_participants']

### Use pandas to load csv converted DataFrame into database

In [22]:
new_snap_df.to_sql(name='snap_participants', con=engine, if_exists='replace', index=False)

pd.read_sql_query('select * from snap_participants', con=engine).head()

,CODEF,STATE_CD,POP10,NUMPOV10,PRGNUM10,PRGBEN10
0,CEN_BEA,NE,25.291,1.787,1.727,2520.0
1,CEN_BEA,NE,2.065,321.000,187.000,272.9
2,CEN_BEA,NE,2.047,191.000,95.000,138.6
3,CEN_BEA,NE,770.000,96.000,68.000,99.2
4,CEN_BEA,NE,9.199,894.000,700.000,1021.4
